In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from data_process import TextEyeTrackingDataset
import numpy as np

# torch.manual_seed(seed)
# np.random.seed(seed)

# 加载数据
dataset = TextEyeTrackingDataset('data/not_skip_data.csv')
batch_size = 1

# 假设dataset是你已经加载的完整数据集
train_size = int(len(dataset) * 0.8)  # 以80%的数据作为训练集
test_size = len(dataset) - train_size  # 剩余20%的数据作为测试集
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [35]:
import torch.nn as nn
from model import MultiModalModelWithAttention
from train_and_evaluate import train_model
import warnings
from sklearn import exceptions

# 忽略特定类型的警告
warnings.filterwarnings("ignore", category=exceptions.UndefinedMetricWarning)


hidden_dim = 128  # LSTM隐藏层维度
output_dim = 1  # 输出维度，二分类问题

model = MultiModalModelWithAttention(hidden_dim, output_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001)

pos_weight = torch.tensor([20])  # 大于1的值，表示正类相对于负类的权重
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)

tensorboard_dir = 'tensorlog/not-skip-attention'
# 训练模型
train_model(model, train_loader, test_loader, optimizer, criterion, device, tensorboard_dir, epochs=30)

-----Epoch 1-------
Train metric in Epoch 1, Loss: 0.012005878612399101, Accuracy: 0.7, AUC: 0.42, Precision: 0.01, Recall: 0.12, F1-score: 0.02
Test metric in Epoch 1, Loss: 0.04598188400268555, Accuracy: 0.5, AUC: 0.74, Precision: 0.05, Recall: 1.0, F1-score: 0.1
-----Epoch 2-------
Train metric in Epoch 2, Loss: 0.014055262319743633, Accuracy: 0.9, AUC: 0.65, Precision: 0.24, Recall: 0.36, F1-score: 0.29
Test metric in Epoch 2, Loss: 0.04577644541859627, Accuracy: 0.79, AUC: 0.41, Precision: 0.0, Recall: 0.0, F1-score: 0
-----Epoch 3-------
Train metric in Epoch 3, Loss: 0.009375122375786304, Accuracy: 1.0, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 3, Loss: 0.04558242857456207, Accuracy: 0.97, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
-----Epoch 4-------
Train metric in Epoch 4, Loss: 0.011931941844522953, Accuracy: 0.97, AUC: 0.5, Precision: 0.0, Recall: 0.0, F1-score: 0
Test metric in Epoch 4, Loss: 0.04533892497420311, Accuracy: 0.97, AUC: 0

In [ ]:
!tensorboard --logdir='tensorlog/all'